Notes:
- Users will likely not want to be recommended an anime they have already watched
    - Additional filtering will need to be done during inference
- Users might be more emotionally invested in a subsequent season of an anime
    - Additional weight for subsequent season of a series? (may be future work)
- Users likely prefer more current anime
    - Have model favor recent releases
    - https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45530.pdf, section 3.3

In [1]:
# Dataset sources: 
# https://www.kaggle.com/datasets/CooperUnion/anime-recommendations-database
# https://www.kaggle.com/datasets/crazygump/myanimelist-scrappind-a-decade-of-anime

import pandas as pd

df_anime_rating = pd.read_csv('../data/01_raw/anime.csv')
df_anime_meta = pd.read_csv('../data/01_raw/MAL-all-from-winter1917-to-fall2022.csv')
df_user_rating = pd.read_csv('../data/01_raw/rating.csv')

print(df_anime_rating.shape)
display(df_anime_rating.head())

print(df_anime_meta.shape)
display(df_anime_meta.head())

print(df_user_rating.shape)
display(df_user_rating.head())

FileNotFoundError: [Errno 2] No such file or directory: './data/CooperUnion/anime.csv'

- rating of -1 means a user has watched a show, but has not rated it (https://www.kaggle.com/datasets/CooperUnion/anime-recommendations-database/discussion/253814)
- Suggestion: use https://jikan.moe/ OR https://github.com/abhinavk99/jikanpy jinkan API to extract studio, producer ()information? (note: rate limited to 60 request per minute)
- Use https://github.com/QasimK/mal-scraper to scrape?
- Scrapy (https://scrapy.org/) can be used to easily extract synopsis, reviews and image links
- https://www.kaggle.com/datasets/crazygump/myanimelist-scrappind-a-decade-of-anime/data can be merged on MAL_id 
- one hot encode genre
- VAE encoding for features?
- NCF for recommender system? https://medium.com/data-science-in-your-pocket/recommendation-systems-using-neural-collaborative-filtering-ncf-explained-with-codes-21a97e48a2f7

In [ ]:
df_anime_meta.rename(columns={"MAL_id":"anime_id"}, inplace=True)
df_anime = df_anime_rating.merge(df_anime_meta, on='anime_id')

# Check for missing values

In [ ]:
df_anime_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [ ]:
df_user_rating['rating'].value_counts()

rating
 8     1646019
-1     1476496
 7     1375287
 9     1254096
 10     955715
 6      637775
 5      282806
 4      104291
 3       41453
 2       23150
 1       16649
Name: count, dtype: int64

In [ ]:
df_anime_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27505 entries, 0 to 27504
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            27505 non-null  object 
 1   MAL_id           27505 non-null  int64  
 2   type             27505 non-null  object 
 3   studio           27505 non-null  object 
 4   release-season   27505 non-null  object 
 5   release-year     27505 non-null  float64
 6   release-date     0 non-null      float64
 7   source-material  22842 non-null  object 
 8   genres           27505 non-null  object 
 9   themes           27505 non-null  object 
 10  demographics     6425 non-null   object 
 11  episodes         27505 non-null  float64
 12  score            27505 non-null  float64
 13  members          27505 non-null  float64
dtypes: float64(5), int64(1), object(8)
memory usage: 2.9+ MB


In [ ]:
for col in df_anime_meta.columns:
    print(df_anime_meta[col].value_counts())

title
Sazae-san                                            213
Sore Ike! Anpanman                                   137
Crayon Shin-chan                                     123
Nintama Rantarou                                     119
Chibi Maruko-chan (1995)                             112
                                                    ... 
Ice Movie                                              1
Gegege no Kitarou: Nippon Bakuretsu                    1
Eiga! Tamagotchi Uchuu Ichi Happy na Monogatari!?      1
Da, da Ge da Xigua                                     1
Big Mac to, Susume                                     1
Name: count, Length: 16858, dtype: int64
MAL_id
2406     213
1960     137
966      123
1199     119
6149     112
        ... 
9053       1
8145       1
6517       1
44517      1
53731      1
Name: count, Length: 16858, dtype: int64
type
TV (Continuing)    9993
TV (New)           5269
OVA                3967
Movie              2931
ONA                2916
Special   